In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_boston
#from sklearn.model_selection import cross_val_predict, cross_validate, cross_val_score, train_test_split, KFold
from sklearn.model_selection import *
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from itertools import combinations
import recordlinkage
from recordlinkage.datasets import *
%matplotlib inline

# Project 1

## Problem statement:
Identify duplicated customers/accounts among multiple data sources. For example we have same customer but under different names in SalesForce, Finance system, CRM system.It is very difficult to get a meaningful analysis for our sales team, finance team, marketing team.
    
    
## Data
This is a common issues with medical care for the healthy department and government as well. I will use the data publised by Freely Extensible Biomedical Record Linkage (FEBRL) project.
      
http://recordlinkage.readthedocs.io/en/latest/ref-datasets.html

http://users.cecs.anu.edu.au/~Peter.Christen/Febrl/febrl-0.3/febrldoc-0.3/manual.html  
    
    
    
## Hypothesis
There are duplicated records in the data sources. The expected result would be any duplicated record by removed or linked.
    
## Potential API or Lib can help

- sklearn
- Record linkage toolkit (used this before)
- Levenshtin distance (used fuzzyWuzzy before)
- Builclassifier dedup lib
- Nazca lib




In [10]:
# EDA
# http://recordlinkage.readthedocs.io/en/latest/ref-datasets.html
"""This data set contains 5000 records (2000 originals and 3000 duplicates), with a maximum of 5 duplicates based on 
one original record (and a Zipf distribution of duplicate records). Distribution of duplicates: 168 originals records 
have 5 duplicate records 161 originals records have 4 duplicate records 212 originals records have 3 duplicate records 
256 originals records have 2 duplicate records 368 originals records have 1 duplicate record 1835 originals records have 
no duplicate record"""

df= load_febrl3()
df.head(2)


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1496-org,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,19560409,1804974
rec-552-dup-3,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,19080419,6089216


In [19]:
print("The features are:")
[x for x in df.columns[0:-1]]

The features are:


['given_name',
 'surname',
 'street_number',
 'address_1',
 'address_2',
 'suburb',
 'postcode',
 'state',
 'date_of_birth']

In [13]:
print("The target is {}".format(df.columns[-1]))

The target is soc_sec_id


In [18]:
# Indexation step
pcl = recordlinkage.BlockIndex(on='given_name')
pairs = pcl.index(df)

# Comparison step
compare_cl = recordlinkage.Compare()

compare_cl.exact('given_name', 'given_name', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(pairs, df)

# Classification step
matches = features[features.sum(axis=1) > 3]

print(matches.head(3))
print(len(matches))

                             given_name  surname  date_of_birth  suburb  \
rec_id         rec_id                                                     
rec-1740-dup-2 rec-1740-org           1      1.0              1       1   
rec-1596-dup-0 rec-1596-org           1      1.0              1       0   
rec-1725-dup-4 rec-1725-org           1      1.0              1       0   

                             state  address_1  
rec_id         rec_id                          
rec-1740-dup-2 rec-1740-org      1        1.0  
rec-1596-dup-0 rec-1596-org      1        0.0  
rec-1725-dup-4 rec-1725-org      1        0.0  
3483


# Project 2

## Problem statement:
Identify whether a potential customer will churn when their current contract is ended. It is more expensive to acquire a new customer than to keep existing one from leaving.I am hoping to use this model to build useful tool to predict customer life time value, help sales team to allocate resource to prevent customer leaving.
    
    
    
## Data
Using the telecom churn data provided by kaggle
    
https://www.kaggle.com/becksddf/churn-in-telecoms-dataset/version/1.
    
https://bigml.com/user/francisco/gallery/dataset/5163ad540c0b5e5b22000383
    
    
## Hypothesis
Predict whether a given customer will churn or retain based on all the features.
    
## Potential API or Lib can help
- sklearn
- ...
- Telecom Churn Notebook https://www.kaggle.com/arpitharavi/telecom-churn-notebook





In [ ]:
#EDA
tele = pd.read_csv('./data/bigml.csv')
tele.head(2)


In [ ]:
print("The features are:")
[x for x in tele.columns[:-1]]


In [ ]:
print("The target is {}".format(tele.columns[-1]))

In [ ]:
tele.describe()